## Set up environment

In [1]:
!rm -r transformers
!git clone -b upload_detr_no_timm https://github.com/nielsrogge/transformers.git
!cd transformers
!pip install -q ./transformers

rm: cannot remove 'transformers': No such file or directory
Cloning into 'transformers'...
remote: Enumerating objects: 210575, done.
remote: Counting objects: 100% (570/570), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 210575 (delta 353), reused 476 (delta 290), pack-reused 210005
Receiving objects: 100% (210575/210575), 180.59 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (157953/157953), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.3 MB/s eta 0:00:00


In [2]:
!pip install -q pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 59.0 MB/s eta 0:00:00


In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00


In [117]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch

preprocessor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')



/usr/local/lib/python3.10/dist-packages/transformers/models/detr/image_processing_detr.py:780: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(


In [118]:
from sklearn.cluster import KMeans
from scipy.sparse import csc_matrix, csr_matrix
import numpy as np


def weight_clustering_kmeans(emodel, bits=5):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  clust_model = emodel
  with torch.no_grad():
    for name, params in clust_model.backbone.named_parameters():
      if 'weight' in name:
        weights = params.cpu().numpy()
        params_shape = weights.shape
        if len(params_shape) == 4:
          weights = weights.reshape([params_shape[0] * params_shape[2], params_shape[1] * params_shape[3]])
        elif len(params_shape) == 1:
          weights = weights.reshape([params_shape[0] // 2, 2])
        shape = weights.shape
        mat = csr_matrix(weights) if shape[0] < shape[1] else csc_matrix(weights)
        min_ = min(mat.data)
        max_ = max(mat.data)
        space = np.linspace(min_, max_, num=2**bits)
        kmeans = KMeans(n_clusters=len(space), init=space.reshape(-1,1), n_init=1)
        kmeans.fit(mat.data.reshape(-1,1))
        new_weight = kmeans.cluster_centers_[kmeans.labels_]
        mat.data = new_weight
        params.data = torch.from_numpy(mat.toarray().reshape(params_shape)).to(device)
        print(name, ' - clustered!')
  return clust_model


In [119]:
model_kmeans = weight_clustering_kmeans(model.model)

conv_encoder.model.conv1.weight  - clustered!
conv_encoder.model.layer1.0.conv1.weight  - clustered!
conv_encoder.model.layer1.0.conv2.weight  - clustered!
conv_encoder.model.layer1.0.conv3.weight  - clustered!
conv_encoder.model.layer1.0.downsample.0.weight  - clustered!
conv_encoder.model.layer1.0.downsample.1.weight  - clustered!
conv_encoder.model.layer1.1.conv1.weight  - clustered!
conv_encoder.model.layer1.1.conv2.weight  - clustered!
conv_encoder.model.layer1.1.conv3.weight  - clustered!
conv_encoder.model.layer1.2.conv1.weight  - clustered!
conv_encoder.model.layer1.2.conv2.weight  - clustered!
conv_encoder.model.layer1.2.conv3.weight  - clustered!
conv_encoder.model.layer2.0.conv1.weight  - clustered!
conv_encoder.model.layer2.0.conv2.weight  - clustered!
conv_encoder.model.layer2.0.conv3.weight  - clustered!
conv_encoder.model.layer2.0.downsample.0.weight  - clustered!
conv_encoder.model.layer2.0.downsample.1.weight  - clustered!
conv_encoder.model.layer2.1.conv1.weight  - cl

In [121]:
def model_size(emodel):
    """
    Return model size in MB.
    """
    return sum(
        p.numel() * p.element_size()
        for p in emodel.parameters()
    ) / 1024**2

print(model_size(model_kmeans))

157.837646484375


In [122]:
detr = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

In [123]:
layer1 = next(model_kmeans.backbone.named_parameters())

In [124]:
detr_layer1 = next(detr.model.backbone.named_parameters())

In [125]:
detr_layer1[1].data.shape

torch.Size([64, 3, 7, 7])

In [126]:
layer1[1].data.cpu().numpy().shape

(64, 3, 7, 7)

In [127]:
model.model = model_kmeans

## Evaluate the model


In [134]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar

!tar -xf ./VOCtrainval_11-May-2012.tar



--2023-10-15 12:20:32--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar.1’

VOCtrainval_11-May- 100%[===================>]   1.86G  26.1MB/s    in 78s     

2023-10-15 12:21:50 (24.3 MB/s) - ‘VOCtrainval_11-May-2012.tar.1’ saved [1999639040/1999639040]



In [129]:
import os

image_sets_path = './VOCdevkit/VOC2012/ImageSets/Main/'
train_path = os.path.join(image_sets_path, 'train.txt')
val_path = os.path.join(image_sets_path, 'val.txt')

images_path = './VOCdevkit/VOC2012/JPEGImages/'
annots_path = './VOCdevkit/VOC2012/Annotations/'

In [130]:
import utils

val_images = utils.read_set(val_path)
train_images = utils.read_set(train_path)

In [132]:
#clustered model metrics

results = utils.evaluate_over_voc(images_path,
                            annots_path,
                            val_images,
                            model,
                            preprocessor,
                            'cuda'
                            )

  0%|          | 0/5823 [00:00<?, ?it/s]

In [133]:
results

({'map': tensor(0.4856),
  'map_50': tensor(0.6441),
  'map_75': tensor(0.5257),
  'map_small': tensor(0.1635),
  'map_medium': tensor(0.3530),
  'map_large': tensor(0.5717),
  'mar_1': tensor(0.4133),
  'mar_10': tensor(0.5443),
  'mar_100': tensor(0.5459),
  'mar_small': tensor(0.2177),
  'mar_medium': tensor(0.4103),
  'mar_large': tensor(0.6190),
  'map_per_class': tensor(-1.),
  'mar_100_per_class': tensor(-1.),
  'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19], dtype=torch.int32)},
 0.078)

In [135]:
#detr (non-clustered) model metrics

results_detr = utils.evaluate_over_voc(images_path,
                            annots_path,
                            val_images,
                            detr,
                            preprocessor,
                            'cuda'
                            )

  0%|          | 0/5823 [00:00<?, ?it/s]

In [136]:
results_detr

({'map': tensor(0.5615),
  'map_50': tensor(0.7287),
  'map_75': tensor(0.6085),
  'map_small': tensor(0.1906),
  'map_medium': tensor(0.4113),
  'map_large': tensor(0.6619),
  'mar_1': tensor(0.4642),
  'mar_10': tensor(0.6254),
  'mar_100': tensor(0.6279),
  'mar_small': tensor(0.2522),
  'mar_medium': tensor(0.4812),
  'mar_large': tensor(0.7130),
  'map_per_class': tensor(-1.),
  'mar_100_per_class': tensor(-1.),
  'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19], dtype=torch.int32)},
 0.077)

In [137]:
model.save_pretrained('clustered_detr')